In [5]:
import json
import pandas as pd
from kafka import KafkaConsumer
import plotly.express as px

consumer = KafkaConsumer(
    'darooghe.top_merchants',
    bootstrap_servers='localhost:9092',
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    group_id='visualization-group-merchant',
    consumer_timeout_ms=15000,
    value_deserializer=lambda m: json.loads(m.decode('utf-8'))
)

data = []
print("Collecting top merchants data...")

for i, msg in enumerate(consumer):
    value = msg.value
    print(f"[{i}] Raw Message:", value)

    try:
        merchant_id = value["merchant_id"]
        transaction_count = int(value["transaction_count"])

        data.append({
            "merchant_id": merchant_id,
            "transaction_count": transaction_count
        })
    except Exception as e:
        print(f"Error parsing message #{i}: {e}")
        continue

consumer.close()

if not data:
    print("No data received from Kafka topic.")
else:
    df = pd.DataFrame(data)
    df = df.groupby("merchant_id").sum().reset_index()

    df = df.sort_values(by="transaction_count", ascending=False).head(5)

    fig = px.bar(
        df,
        x="merchant_id",
        y="transaction_count",
        title="Top 5 Merchants by Transaction Count",
        labels={"merchant_id": "Merchant ID", "transaction_count": "Transaction Count"}
    )

    fig.update_layout(
        xaxis_title='Merchant ID',
        yaxis_title='Number of Transactions',
        template='plotly_dark'
    )

    fig.show()


[0] Raw Message: {'merchant_id': 'merch_4', 'transaction_count': 277}
[1] Raw Message: {'merchant_id': 'merch_28', 'transaction_count': 270}
[2] Raw Message: {'merchant_id': 'merch_10', 'transaction_count': 268}
[3] Raw Message: {'merchant_id': 'merch_44', 'transaction_count': 268}
[4] Raw Message: {'merchant_id': 'merch_6', 'transaction_count': 268}
[5] Raw Message: {'merchant_id': 'merch_4', 'transaction_count': 285}
[6] Raw Message: {'merchant_id': 'merch_6', 'transaction_count': 278}
[7] Raw Message: {'merchant_id': 'merch_44', 'transaction_count': 277}
[8] Raw Message: {'merchant_id': 'merch_10', 'transaction_count': 277}
[9] Raw Message: {'merchant_id': 'merch_23', 'transaction_count': 271}
[10] Raw Message: {'merchant_id': 'merch_4', 'transaction_count': 289}
[11] Raw Message: {'merchant_id': 'merch_6', 'transaction_count': 285}
[12] Raw Message: {'merchant_id': 'merch_44', 'transaction_count': 282}
[13] Raw Message: {'merchant_id': 'merch_10', 'transaction_count': 280}
[14] Raw